In [1]:
url = "https://www.tennisabstract.com/charting/20251113-M-Tour_Finals-RR-Carlos_Alcaraz-Lorenzo_Musetti.html"

In [2]:
from selenium import webdriver
driver = webdriver.Chrome()

In [3]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import time # 待機用
import re

In [19]:
driver.get(url)

In [23]:
id = "pointlog"
button_selector = (By.ID, id)

In [24]:
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(button_selector)
)
print("ボタンの要素が見つかりました。")

ボタンの要素が見つかりました。


In [25]:

# 目的のボタンをクリック
button = driver.find_element(*button_selector)
button.click()
print("ボタンをクリックしました。")

ボタンをクリックしました。


In [26]:
page_source = driver.page_source

In [27]:
tables = pd.read_html(page_source)

/var/folders/3s/sh8zstl54dl_t2y55d7th57m0000gn/T/ipykernel_11614/3904309482.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(page_source)


In [41]:
df = tables[6]

In [66]:
df

,Server,Sets,Games,Points,info
0,Carlos Alcaraz,0‑0,0‑0,0‑0,"1st serve down the T, service winner."
1,Carlos Alcaraz,0‑0,0‑0,15‑0,"1st serve down the T, fault (wide). 2nd serve ..."
2,Carlos Alcaraz,0‑0,0‑0,15‑15,"1st serve down the T, service winner."
3,Carlos Alcaraz,0‑0,0‑0,30‑15,1st serve down the T; forehand chip/slice retu...
4,Carlos Alcaraz,0‑0,0‑0,40‑15,"1st serve wide, service winner."
...,...,...,...,...,...
117,Carlos Alcaraz,1‑0,5‑1,15‑30,1st serve wide; backhand slice return down the...
118,Carlos Alcaraz,1‑0,5‑1,30‑30,1st serve wide; forehand return down the line ...
119,Carlos Alcaraz,1‑0,5‑1,40‑30,1st serve wide; backhand slice return down the...
120,Carlos Alcaraz,1‑0,5‑1,40‑40,"1st serve down the T, fault (long). 2nd serve ..."


In [65]:
df.rename(columns={df.columns[4]: 'info'}, inplace=True)

In [ ]:
def find_serve_patterns(info: str) -> tuple[str | None, str | None]:
    """
    Find serve patterns in the given info string.
    
    Args:
        info (str)
    Returns:
        tuple for first and second serve.
        T represents "serve down the T"
        W represents "serve wide"
        B represents "serve to body"
    """
    patterns = {
        "serve down the T": "T",
        "serve wide": "W",
        "serve to body": "B"
    }
    search_pattern = "|".join(re.escape(p) for p in patterns.keys())
    found_substrings = re.findall(search_pattern, info)
    
    results = [patterns[sub] for sub in found_substrings]
    
    first = results[0] if len(results) >= 1 else None
    second = results[1] if len(results) >= 2 else None

    return first, second

In [52]:
def find_fault_patterns(info: str) -> tuple[str | None, str | None]:
    """
    Find fault patterns in the given info string.

    Args:
        info (str)
    Returns:
        tuple for first and second fault.
        L represents long
        W represents wide
        N represents net
    """
    patterns = {
        "fault (long)": "L",
        "fault (wide)": "W",
        "fault (net)": "N"
    }
    search_pattern = "|".join(re.escape(p) for p in patterns.keys())
    found_substrings = re.findall(search_pattern, info)
    
    results = [patterns[sub] for sub in found_substrings]
    
    first = results[0] if len(results) >= 1 else None
    second = results[1] if len(results) >= 2 else None

    return first, second

In [60]:
info = df.iloc[3, 4]

In [61]:
info

'1st serve down the T; forehand chip/slice return down the middle (shallow); forehand crosscourt, winner.'

In [62]:
print(find_serve_patterns(info))
print(find_fault_patterns(info))

('T', None)
(None, None)


In [69]:
def info_parser(info):
    if not isinstance(info, str):
        return [None, None, None, None]
    serves = find_serve_patterns(info)
    faults = find_fault_patterns(info)
    return [serves[0], faults[0], serves[1], faults[1]]

In [64]:
info_parser(info)

['T', None, None, None]

In [70]:
service_columns = ['first', 'first_f', 'second', 'second_f']
df[service_columns] = df['info'].apply(info_parser).apply(pd.Series)

In [74]:
df[
    df[service_columns].isna().all(axis=1)
]

,Server,Sets,Games,Points,info,first,first_f,second,second_f
5,NaN,NaN,NaN,NaN,NaN,None,None,None,None
11,NaN,NaN,NaN,NaN,NaN,None,None,None,None
17,NaN,NaN,NaN,NaN,NaN,None,None,None,None
26,NaN,NaN,NaN,NaN,NaN,None,None,None,None
31,NaN,NaN,NaN,NaN,NaN,None,None,None,None
36,NaN,NaN,NaN,NaN,NaN,None,None,None,None
43,NaN,NaN,NaN,NaN,NaN,None,None,None,None
48,NaN,NaN,NaN,NaN,NaN,None,None,None,None
54,NaN,NaN,NaN,NaN,NaN,None,None,None,None
69,NaN,NaN,NaN,NaN,NaN,None,None,None,None


In [76]:
df.head(10)

,Server,Sets,Games,Points,info,first,first_f,second,second_f
0,Carlos Alcaraz,0‑0,0‑0,0‑0,"1st serve down the T, service winner.",T,None,None,None
1,Carlos Alcaraz,0‑0,0‑0,15‑0,"1st serve down the T, fault (wide). 2nd serve ...",T,W,W,None
2,Carlos Alcaraz,0‑0,0‑0,15‑15,"1st serve down the T, service winner.",T,None,None,None
3,Carlos Alcaraz,0‑0,0‑0,30‑15,1st serve down the T; forehand chip/slice retu...,T,None,None,None
4,Carlos Alcaraz,0‑0,0‑0,40‑15,"1st serve wide, service winner.",W,None,None,None
5,NaN,NaN,NaN,NaN,NaN,None,None,None,None
6,Lorenzo Musetti,0‑0,0‑1,0‑0,1st serve down the T; backhand return down the...,T,None,None,None
7,Lorenzo Musetti,0‑0,0‑1,15‑0,1st serve wide; backhand return crosscourt (de...,W,None,None,None
8,Lorenzo Musetti,0‑0,0‑1,15‑15,"1st serve wide, service winner.",W,None,None,None
9,Lorenzo Musetti,0‑0,0‑1,30‑15,"1st serve down the T, service winner.",T,None,None,None


In [77]:
df["Points"]

0        0‑0
1       15‑0
2      15‑15
3      30‑15
4      40‑15
       ...  
117    15‑30
118    30‑30
119    40‑30
120    40‑40
121    AD‑40
Name: Points, Length: 122, dtype: object

In [113]:
import pandas as pd
import numpy as np

def score_to_left_win(score_series: pd.Series) -> pd.Series:
    """
    テニスのスコア系列から、各ポイントでレフト側が勝ったかどうかを判定します。
    NaNはセット終了を意味し、直前のポイントがゲーム終了点だった場合はTrueを返します。
    
    Args:
        score_series: '0-0', '15-0', 'NaN' などの文字列を含むSeries。
        
    Returns:
        レフト側がポイントを獲得した場合は True、それ以外は False のブール値Series。
    """
    
    # 1. スコアを分割し、PandasのDataFrameに格納
    # Shift(-1)を使用して、各行に「次のスコア」を併記します
    scores_df = pd.DataFrame({
        'current_score': score_series,
        'next_score': score_series.shift(-1)
    })
    # スコアのスペースを削除
    scores_df['current_score'] = scores_df['current_score'].str.replace(' ', '', regex=False).str.replace('‑', '-', regex=False)
    scores_df['next_score'] = scores_df['next_score'].str.replace(' ', '', regex=False).str.replace('‑', '-', regex=False)
    
    score_map = {'0': 0, '15': 1, '30': 2, '40': 3, 'AD': 4}

    def is_left_win(current_score, next_score):
        """
        現在のスコアと次のスコアを比較し、レフト側が勝ったか（またはセットが終了したか）を判定する内部関数。
        """
        #　空行(セット間)の場合はNoneを返す
        if pd.isna(current_score):
            return None
        # セット終了時
        if pd.isna(next_score):
            if current_score == '40-0' or \
               current_score == '40-15' or \
               current_score == '40-30' or \
               current_score == 'AD-40':
                return True # レフト側の勝利でゲーム終了
            elif current_score == '0-40' or \
                 current_score == '15-40' or \
                 current_score == '30-40' or \
                 current_score == '40-AD':
                return False # ライト側の勝利でゲーム終了
            else:
                return None # error case

        # 通常ポイント
        try:
            L_curr, R_curr = current_score.split('-')
            L_next, R_next = next_score.split('-')
            
            # 数値に変換 (存在しないスコア表記は例外でFalse)
            L_curr_val = score_map.get(L_curr)
            R_curr_val = score_map.get(R_curr)
            L_next_val = score_map.get(L_next)
            R_next_val = score_map.get(R_next)

            # いずれかの値がNoneなら不正なスコアとしてFalse
            if None in [L_curr_val, R_curr_val, L_next_val, R_next_val]:
                return "None exists"
            
            diff_L = L_next_val - L_curr_val
            diff_R = R_next_val - R_curr_val
            
            # 通常のポイント増加
            if diff_L == 1 and diff_R == 0:
                return True
            # アドバンテージの解消
            if current_score == '40-AD' and next_score == '40-40':
                return True
            # その他(diff_R == 1, AD-40 -> 40-40, 40-40 -> 40-AD) はライトの勝利なので False
            return False

        except ValueError:
            return "fail to split or convert"
        except Exception:
            return "unexpected error"

    result_series = scores_df.apply(
        lambda row: is_left_win(row['current_score'], row['next_score']), 
        axis=1
    )
    return result_series

In [115]:
scores = pd.DataFrame({"raw": df["Points"],
                      "convert": score_to_left_win(df['Points'])})

In [121]:
df["left_win"] = score_to_left_win(df['Points'])

In [122]:
df

,Server,Sets,Games,Points,info,first,first_f,second,second_f,left_win
0,Carlos Alcaraz,0‑0,0‑0,0‑0,"1st serve down the T, service winner.",T,None,None,None,True
1,Carlos Alcaraz,0‑0,0‑0,15‑0,"1st serve down the T, fault (wide). 2nd serve ...",T,W,W,None,False
2,Carlos Alcaraz,0‑0,0‑0,15‑15,"1st serve down the T, service winner.",T,None,None,None,True
3,Carlos Alcaraz,0‑0,0‑0,30‑15,1st serve down the T; forehand chip/slice retu...,T,None,None,None,True
4,Carlos Alcaraz,0‑0,0‑0,40‑15,"1st serve wide, service winner.",W,None,None,None,True
...,...,...,...,...,...,...,...,...,...,...
117,Carlos Alcaraz,1‑0,5‑1,15‑30,1st serve wide; backhand slice return down the...,W,None,None,None,True
118,Carlos Alcaraz,1‑0,5‑1,30‑30,1st serve wide; forehand return down the line ...,W,None,None,None,True
119,Carlos Alcaraz,1‑0,5‑1,40‑30,1st serve wide; backhand slice return down the...,W,None,None,None,False
120,Carlos Alcaraz,1‑0,5‑1,40‑40,"1st serve down the T, fault (long). 2nd serve ...",T,L,T,None,True


In [4]:
from main import parse_tennis_table

In [1]:
url = "https://www.tennisabstract.com/charting/20251113-M-Tour_Finals-RR-Carlos_Alcaraz-Lorenzo_Musetti.html"

In [5]:
parse_tennis_table(url, file_name="test_output_same_column.csv")

Accessing URL: https://www.tennisabstract.com/charting/20251113-M-Tour_Finals-RR-Carlos_Alcaraz-Lorenzo_Musetti.html
Expanding table on URL: https://www.tennisabstract.com/charting/20251113-M-Tour_Finals-RR-Carlos_Alcaraz-Lorenzo_Musetti.html
Reading table from URL: https://www.tennisabstract.com/charting/20251113-M-Tour_Finals-RR-Carlos_Alcaraz-Lorenzo_Musetti.html


/Users/meina/Github/meina-t/parse_tennis_score/main.py:53: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(page_source)


In [6]:
df_n

NameError: name 'df_n' is not defined